# Mecanismo de Busca Semântica para Perguntas Médicas

In [2]:
# Importar dependencias
import kagglehub
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

## Passo 1: Configuração e Carregamento de Dados

In [ ]:
print("Baixando o conjunto de dados MedQuad do KaggleHub...")
try:
    path = kagglehub.dataset_download("pythonafroz/medquad-medical-question-answer-for-ai-research")
    print(f"Dataset baixado com sucesso em: {path}")
except Exception as e:
    print(f"Erro ao baixar o dataset. Verifique sua autenticação do Kaggle. Erro: {e}")
    exit()

print("Carregando o arquivo CSV para um DataFrame pandas...")
# Carrega o dataset
try:
    df = pd.read_csv(f"{path}/medquad.csv")
    # Limpeza básica: remove linhas onde a pergunta ou a resposta estão vazias
    df.dropna(subset=['question', 'answer'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Dataset carregado. Número de pares de pergunta-resposta: {len(df)}")
except FileNotFoundError:
    print(f"Arquivo medquad.csv não encontrado no caminho: {path}")
    exit()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilizando o dispositivo: {device}")

print("Carregando o modelo e o tokenizador da Hugging Face...")
# Modelo pré-treinado para a língua inglesa
nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo).to(device)

Baixando o conjunto de dados MedQuad do KaggleHub...
Dataset baixado com sucesso em: /home/codespace/.cache/kagglehub/datasets/pythonafroz/medquad-medical-question-answer-for-ai-research/versions/1
Carregando o arquivo CSV para um DataFrame pandas...
Dataset carregado. Número de pares de pergunta-resposta: 16407
Utilizando o dispositivo: cpu
Carregando o modelo e o tokenizador da Hugging Face...


## Passo 2: Função para Geração de Embeddings (Mean Pooling)

In [ ]:
def generate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    attention_mask = inputs['attention_mask']
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_states.size()).float()
    sum_embeddings = torch.sum(last_hidden_states * mask_expanded, 1)
    sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
    
    mean_pooled = sum_embeddings / sum_mask
    
    return F.normalize(mean_pooled, p=2, dim=1)

## Passo 3: Indexação - Gerando Embeddings para 1/10 das Perguntas do Dataset

In [ ]:
print("\nIniciando a indexação em um subconjunto de 1/10 do dataset...")

subset_size = len(df) // 10
df_subset = df.iloc[:subset_size].copy()

print(f"Dataset original: {len(df)} perguntas. Usando um subconjunto de {len(df_subset)} perguntas.")

print("\n--- Exemplo das 20 primeiras perguntas do subconjunto ---") # imprimir perguntas para realizar testes posteriormente
for i, pergunta in enumerate(df_subset['question'].head(20)):
    print(f"{i+1}. {pergunta}")
print("-" * 50)

questions_list = df_subset['question'].tolist()

batch_size = 32
all_embeddings = []

# Loop para processar a lista em lotes
for i in range(0, len(questions_list), batch_size):
    batch_texts = questions_list[i:i + batch_size]
    
    batch_embeddings = generate_embeddings(batch_texts)
    
    all_embeddings.append(batch_embeddings.cpu()) # Move para a CPU para evitar sobrecarga de memória da GPU
    
    print(f"Processando lote {i//batch_size + 1}/{(len(questions_list) - 1)//batch_size + 1}")

dataset_embeddings = torch.cat(all_embeddings, dim=0)

print("Indexação do subconjunto concluída com sucesso!")
print(f"Formato do tensor final de embeddings: {dataset_embeddings.shape}")


Iniciando a indexação em um subconjunto de 1/10 do dataset...
Dataset original: 16407 perguntas. Usando um subconjunto de 1640 perguntas.

--- Exemplo das 20 primeiras perguntas do subconjunto ---
1. What is (are) Glaucoma ?
2. What causes Glaucoma ?
3. What are the symptoms of Glaucoma ?
4. What are the treatments for Glaucoma ?
5. What is (are) Glaucoma ?
6. What is (are) Glaucoma ?
7. What is (are) Glaucoma ?
8. Who is at risk for Glaucoma? ?
9. How to prevent Glaucoma ?
10. What are the symptoms of Glaucoma ?
11. What are the treatments for Glaucoma ?
12. what research (or clinical trials) is being done for Glaucoma ?
13. Who is at risk for Glaucoma? ?
14. What is (are) Glaucoma ?
15. What is (are) High Blood Pressure ?
16. What causes High Blood Pressure ?
17. Who is at risk for High Blood Pressure? ?
18. How to prevent High Blood Pressure ?
19. What are the symptoms of High Blood Pressure ?
20. What is (are) High Blood Pressure ?
-------------------------------------------------

## Passo 4: Função de Busca Semântica

In [ ]:
def find_best_answer(user_question, dataset_embeddings, data_frame):
    query_embedding = generate_embeddings([user_question]).cpu()

    cosine_scores = torch.matmul(query_embedding, dataset_embeddings.T)

    best_match_index = torch.argmax(cosine_scores).item()

    best_question = data_frame.iloc[best_match_index]['question']
    best_answer = data_frame.iloc[best_match_index]['answer']
    similarity_score = cosine_scores[0][best_match_index].item()

    return best_question, best_answer, similarity_score

In [ ]:
print("--- Mecanismo de Busca Semântica Médica ---")
print("Digite sua pergunta em inglês. Digite 'sair' para terminar.")

while True:
    user_query = input("\nDigite 'sair' para encerrar a aplicação\nSua pergunta: ")
    
    if user_query.lower() == 'sair':
        break

    # Busca a resposta
    similar_q, answer, score = find_best_answer(user_query, dataset_embeddings, df)

    # Exibe os resultados
    print("-" * 50)
    print(f"Pergunta do Usuário: {user_query}")
    print("-" * 50)
    print(f"Pergunta Mais Similar Encontrada (Score: {score:.4f}): {similar_q}")
    print("\nResposta Encontrada:")
    print(answer)
    print("-" * 50)

--- Mecanismo de Busca Semântica Médica ---
Digite sua pergunta em inglês. Digite 'sair' para terminar.
--------------------------------------------------
Pergunta do Usuário: risk glaucoma
--------------------------------------------------
Pergunta Mais Similar Encontrada (Score: 0.8322): How to prevent Glaucoma ?

Resposta Encontrada:
At this time, we do not know how to prevent glaucoma. However, studies have shown that the early detection and treatment of glaucoma, before it causes major vision loss, is the best way to control the disease. So, if you fall into one of the higher risk groups for the disease, make sure to have a comprehensive dilated eye exam at least once every one to two years.  Get tips on finding an eye care professional. Learn what a comprehensive dilated eye exam involves.
--------------------------------------------------
--------------------------------------------------
Pergunta do Usuário: prevent blood pressure
-----------------------------------------------